In [35]:
import sys
from pathlib import Path

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

In [36]:
%reload_ext autoreload
%autoreload 2

(opcional) Este código permite conocer lo que hay en hopsworks

In [37]:
# Conocer que datos hay en el feature store
import hopsworks
import pandas as pd
import config

FG_NAME = "times_series_bolleria_feature_group"
FV_NAME = "times_series_bolleria_feature_view"
FV_VERSION = 1

# Conectar con el proyecto de Hopsworks y al feature store
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)


# Obtener el feature store
feature_store = project.get_feature_store()

# Listar los feature groups
feature_groups = feature_store.get_feature_groups(name=FG_NAME)
print(f"Feature groups encontrados: {len(feature_groups)}")
for fg in feature_groups:
    print(f" - {fg.name}")



2025-08-22 16:46:22,680 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-22 16:46:22,684 INFO: Initializing external client
2025-08-22 16:46:22,685 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-08-22 16:46:22,684 INFO: Initializing external client
2025-08-22 16:46:22,685 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-08-22 16:46:23,807 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
Feature groups encontrados: 1
 - times_series_bolleria_feature_group
Feature groups encontrados: 1
 - times_series_bolleria_feature_group


In [38]:
# Para cada FG, saca numero de filas y rango de timestamp
for fg in feature_groups:
    print(f"Feature group: {fg.name}")
    df = fg.read()
    print(f" - Número de filas: {len(df)}")
    if not df.empty:
        print(f" - Rango de week_start: {df['week_start'].min()} a {df['week_start'].max()}")
    else:
        print(" - No hay datos en este feature group.")

Feature group: times_series_bolleria_feature_group
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00


In [39]:
# Listas los features views
feature_views = feature_store.get_feature_views(name=FV_NAME)
# Filtrar por versión
feature_views = [fv for fv in feature_views if fv.version == FV_VERSION]
print(f"Feature views encontrados: {len(feature_views)}")
for fv in feature_views:
    print(f" - {fv.name}")
    print(f" - Version: {fv.version}")

# Para cada feature view saca el número de filas y rango de timestamp
batch_data = []
for fv in feature_views:
    df = fv.get_batch_data()
    batch_data.append(df)

# Imprimir información sobre los datos de cada feature view y las columnas
for i, df in enumerate(batch_data):
    print(f"Feature view {feature_views[i].name}:")
    print(f" - Número de filas: {len(df)}")
    if not df.empty:
        print(f" - Rango de week_start: {df['week_start'].min()} a {df['week_start'].max()}")
        print(f" - Columnas: {list(df.columns)}")
    else:
        print(" - No hay datos en este feature view.")

Feature views encontrados: 1
 - times_series_bolleria_feature_view
 - Version: 1
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 
Feature view times_series_bolleria_feature_view:
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00
 - Columnas: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 
Feature view times_series_bolleria_feature_view:
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00
 - Columnas: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']


# Conexión al proyecto, feature store y model registry en hopsworks

In [40]:
import hopsworks

# Conexión a hopsworks y proyecto   
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)
feature_store = project.get_feature_store()

# Conexión al model registry
model_registry = project.get_model_registry()

print("Conectado a proyecto:", project.name)
print("Conectado al feature store:", feature_store.name)
print("Conectado al model registry:", model_registry)


2025-08-22 16:46:31,319 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-22 16:46:31,325 INFO: Initializing external client
2025-08-22 16:46:31,326 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-08-22 16:46:31,325 INFO: Initializing external client
2025-08-22 16:46:31,326 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-08-22 16:46:32,451 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
Conectado a proyecto: fleca_mlops
Conectado al feature store: fleca_mlops_featurestore
Conectado al model registry: ModelRegistry(project: 'fleca_mlops')
Conectado a proyecto: fleca_mlops
Conectado al feature store: fleca_mlops_featurestore
Conectado al model registry: ModelRegistry(project: 'fleca_mlops')


# Leemos Batch Score del Feature View

In [41]:
from datetime import timedelta
import pandas as pd

# Lectura batch cruda de feature views
feature_views= feature_store.get_feature_view(name=FV_NAME, version=FV_VERSION)
ts_df = feature_views.get_batch_data(
    start_date=None,
    end_date=None,
    features=None,
    )

# Ordenar por la columna de fecha antes de visualizar
ts_df = ts_df.sort_values('week_start').reset_index(drop=True)
ts_df.head(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


,familia,base_imponible,is_summer_peak,is_easter,week_start
0,BOLLERIA,825.11,0,0,2023-01-02 00:00:00+00:00
1,BOLLERIA,658.40,0,0,2023-01-09 00:00:00+00:00
2,BOLLERIA,741.40,0,0,2023-01-16 00:00:00+00:00
3,BOLLERIA,653.64,0,0,2023-01-23 00:00:00+00:00
4,BOLLERIA,680.46,0,0,2023-01-30 00:00:00+00:00


# Añadimos features (lags, variables exógenas) y target

In [42]:
from src.model import transformar_features_target

# Procesar datos usando la función mejorada que acepta tuplas directamente
ts_df = transformar_features_target(
    ts_df,
    lags_list=[1, 2, 3, 52], 
    columna_target='base_imponible',
    cols_exogenas=['is_easter', 'is_summer_peak'],
    periodos_adelante=1,
    eliminar_nulos=True,
    return_format='dataframe'  # Obtenemos un único DataFrame con features y target
)

# Resetear el índice
ts_df = ts_df.reset_index(drop=True)

# Mostrar las primeras filas del DataFrame transformado
ts_df.head(5)

2025-08-22 16:46:38,335 INFO: Usando DataFrame de entrada: (133, 5)
2025-08-22 16:46:38,339 INFO: Retornando DataFrame combinado: (80, 8)
2025-08-22 16:46:38,339 INFO: Retornando DataFrame combinado: (80, 8)


,base_imponible_lag1,base_imponible_lag2,base_imponible_lag3,base_imponible_lag52,is_easter,is_summer_peak,week_start,target
0,572.51,534.79,563.18,825.11,0,0,2024-01-15 00:00:00+00:00,680.30
1,597.65,572.51,534.79,658.40,0,0,2024-01-22 00:00:00+00:00,603.99
2,680.30,597.65,572.51,741.40,0,0,2024-01-29 00:00:00+00:00,600.14
3,603.99,680.30,597.65,653.64,0,0,2024-02-05 00:00:00+00:00,689.32
4,600.14,603.99,680.30,680.46,0,0,2024-02-12 00:00:00+00:00,627.76


# Cargamos el modelo

In [43]:
import joblib
# Cargar el modelo del model registry
## se obtiene del config

MODEL_NAME = "fleca_bolleria_predictor_next_week"
MODEL_VERSION = 1
MODEL_FILE ='xgboost_hopsworks.pkl'


model_registry = project.get_model_registry()
model = model_registry.get_model(name=MODEL_NAME, version=MODEL_VERSION)    
from pathlib import Path

model_dir = model.download()
model = joblib.load(Path(model_dir) / MODEL_FILE)

print(f'Modelo {MODEL_NAME} V{MODEL_VERSION} cargado correctamente')

Downloading: 0.000%|          | 0/78699 elapsed<00:00 remaining<?

Modelo fleca_bolleria_predictor_next_week V1 cargado correctamente


In [44]:
# Verificar qué columnas tenemos después de transformar_features_target
print("Columnas después de transformar_features_target:")
print(ts_df.columns.tolist())

print("\nFeatures que espera el modelo:")
if 'model' in globals():
    print(model.feature_names_in_)
else:
    print("❌ El modelo no está definido. Por favor, ejecuta la celda que carga el modelo antes de continuar.")

print("\nComparación:")
for feature in model.feature_names_in_:
    if feature in ts_df.columns:
        print(f"✅ {feature}: EXISTE")
    else:
        print(f"❌ {feature}: NO EXISTE")
        
# Verificar si existen las versiones con 'target_lag'
target_lags = [col for col in ts_df.columns if 'target_lag' in col]
print(f"\nColumnas target_lag encontradas: {target_lags}")

Columnas después de transformar_features_target:
['base_imponible_lag1', 'base_imponible_lag2', 'base_imponible_lag3', 'base_imponible_lag52', 'is_easter', 'is_summer_peak', 'week_start', 'target']

Features que espera el modelo:
['base_imponible_lag1' 'base_imponible_lag2' 'base_imponible_lag3'
 'base_imponible_lag52' 'is_easter' 'is_summer_peak']

Comparación:
✅ base_imponible_lag1: EXISTE
✅ base_imponible_lag2: EXISTE
✅ base_imponible_lag3: EXISTE
✅ base_imponible_lag52: EXISTE
✅ is_easter: EXISTE
✅ is_summer_peak: EXISTE

Columnas target_lag encontradas: []


In [45]:
model.feature_names_in_


array(['base_imponible_lag1', 'base_imponible_lag2',
       'base_imponible_lag3', 'base_imponible_lag52', 'is_easter',
       'is_summer_peak'], dtype='<U20')

In [46]:
# Limpiar columnas innecesarias
features = ts_df.drop(columns=['week_start'])

# Hacemos predicciones

In [47]:
# Predicción directa para la próxima semana (sin autoregresión)
from datetime import timedelta

print("=== PREDICCIÓN DIRECTA DE LA PRÓXIMA SEMANA ===")

# Tomar la última fila de datos históricos
ultimo_lunes = ts_df['week_start'].max()
last_row = ts_df.iloc[-1].copy()
feature_names = model.feature_names_in_

# Calcular la fecha de la próxima semana
fecha_siguiente = ultimo_lunes + timedelta(days=7)
last_row['week_start'] = fecha_siguiente

# Preparar los features para el modelo
X = last_row[feature_names].values.reshape(1, -1)
prediccion = model.predict(X)[0]

print(f"Fecha predicha: {fecha_siguiente.date()}")
print(f"Predicción base_imponible: {prediccion:.2f}")

# Mostrar resultado en DataFrame
resultado = pd.DataFrame({
    'week_start': [fecha_siguiente],
    'predicted_base_imponible': [prediccion]
})
print(resultado)


=== PREDICCIÓN DIRECTA DE LA PRÓXIMA SEMANA ===
Fecha predicha: 2025-08-11
Predicción base_imponible: 1310.64
                 week_start  predicted_base_imponible
0 2025-08-11 00:00:00+00:00               1310.636108


# Crear el feature group y subir las predicciones

In [53]:
# Subir predicción semanal (variable 'resultado') al feature group y feature view de Hopsworks
from hsfs.feature_group import FeatureGroup
from hsfs.feature_view import FeatureView
import logging

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('feature_group_creation')

PRED_FG_NAME = "bolleria_predicciones_feature_group"
PRED_FG_VERSION = 1

# Usar la variable 'resultado' como DataFrame de predicción
df_pred = resultado.copy()  # Debe tener columnas 'week_start' y 'predicted_base_imponible'

# Mostrar los datos que se van a insertar
print("DataFrame que se va a insertar:")
print(df_pred)

# 1. Primero intentamos crear el feature group explícitamente
try:
    # Si ya existe, esto lanzará una excepción que capturaremos después
    fg_pred = feature_store.create_feature_group(
        name=PRED_FG_NAME,
        version=PRED_FG_VERSION,
        description="Predicciones semanales de base_imponible para bolleria",
        primary_key=["week_start"],
        online_enabled=False,
        statistics_config={"enabled": True},
        event_time='week_start'
    )
    logger.info(f"Feature group '{PRED_FG_NAME}' creado correctamente.")
    
    # Al crear un nuevo feature group, necesitamos hacer insert con el DataFrame
    insert_job = fg_pred.insert(df_pred, write_options={'wait_for_job': True})
    logger.info(f"Datos insertados en el feature group recién creado. Job: {insert_job}")
    
except Exception as create_error:
    logger.warning(f"No se pudo crear el feature group: {create_error}")
    logger.info("Intentando obtener el feature group existente...")
    
    # 2. Si el feature group ya existe, lo obtenemos
    try:
        fg_pred = feature_store.get_feature_group(
            name=PRED_FG_NAME,
            version=PRED_FG_VERSION
        )
        logger.info(f"Feature group existente recuperado: {fg_pred.name} v{fg_pred.version}")
        
        # Insertamos los nuevos datos en el feature group existente
        insert_job = fg_pred.insert(df_pred, write_options={'wait_for_job': True})
        logger.info(f"Datos insertados en el feature group existente. Job: {insert_job}")
        
    except Exception as get_error:
        logger.error(f"Error al recuperar el feature group: {get_error}")
        logger.error("No se pudo ni crear ni recuperar el feature group.")
        import traceback
        traceback.print_exc()
        fg_pred = None

# 3. Verificación final
if fg_pred is not None:
    print(f"✅ Feature group operativo: {fg_pred.name} (versión {fg_pred.version})")
    
    # 4. (Opcional) Crear el feature view si no existe
    try:
        # Primero intentar recuperar
        try:
            fv_pred = feature_store.get_feature_view(name=PRED_FG_NAME, version=PRED_FG_VERSION)
            print(f"✅ Feature view '{PRED_FG_NAME}' ya existe.")
        except:
            # Si no existe, crear uno nuevo
            query = fg_pred.select_all()
            fv_pred = feature_store.create_feature_view(
                name=PRED_FG_NAME,
                version=PRED_FG_VERSION,
                query=query,
                description="Vista de predicciones semanales de bolleria"
            )
            print("✅ Feature view creado correctamente.")
    except Exception as fv_error:
        print(f"⚠️ Error con el feature view: {fv_error}")
else:
    print("❌ Error: No se pudo operar con el feature group de predicciones.")

DataFrame que se va a insertar:
                 week_start  predicted_base_imponible
0 2025-08-11 00:00:00+00:00               1310.636108
2025-08-22 16:53:42,740 INFO: Feature group 'bolleria_predicciones_feature_group' creado correctamente.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1242272/fs/1224799/fg/1504659
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1242272/fs/1224799/fg/1504659


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00



Launching job: bolleria_predicciones_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242272/jobs/named/bolleria_predicciones_feature_group_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242272/jobs/named/bolleria_predicciones_feature_group_1_offline_fg_materialization/executions
2025-08-22 16:53:58,231 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-08-22 16:53:58,231 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-08-22 16:54:04,556 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-08-22 16:54:04,556 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-08-22 16:55:48,890 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Fina

# Verificación del feature group y creación del feature view

Después de crear el feature group, verificamos su existencia y creamos un feature view asociado.

In [54]:
# Verificar que el feature group existe y crear feature view
import logging
from hsfs.feature_group import FeatureGroup
from hsfs.feature_view import FeatureView

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('feature_view_creation')

# Definir constantes
PRED_FG_NAME = "bolleria_predicciones_feature_group"
PRED_FG_VERSION = 1
PRED_FV_NAME = "bolleria_predicciones_feature_view"
PRED_FV_VERSION = 1

# 1. Verificar que el feature group existe
try:
    fg_pred = feature_store.get_feature_group(name=PRED_FG_NAME, version=PRED_FG_VERSION)
    print(f"✅ Feature group encontrado: {fg_pred.name} (versión {fg_pred.version})")
    
    # Mostrar los datos actuales en el feature group
    print("\nDatos actuales en el feature group:")
    df_fg = fg_pred.read()
    print(df_fg)
    
    # 2. Crear feature view
    print("\nCreando feature view...")
    
    try:
        # Primero intentamos recuperar el feature view si existe
        try:
            fv_pred = feature_store.get_feature_view(name=PRED_FV_NAME, version=PRED_FV_VERSION)
            print(f"✅ Feature view existente: {fv_pred.name} (versión {fv_pred.version})")
        except:
            # Si no existe, crear uno nuevo
            query = fg_pred.select_all()  # Crear la query seleccionando todas las columnas
            
            print("Creando nuevo feature view...")
            fv_pred = feature_store.create_feature_view(
                name=PRED_FV_NAME,
                version=PRED_FV_VERSION,
                query=query,
                description="Vista de predicciones semanales de bolleria"
            )
            print(f"✅ Nuevo feature view creado: {fv_pred.name} (versión {fv_pred.version})")
        
        # Verificar datos en el feature view
        print("\nVerificando datos en el feature view:")
        df_fv = fv_pred.get_batch_data()
        print(df_fv)
        
    except Exception as fv_error:
        print(f"❌ Error al crear o recuperar el feature view: {fv_error}")
        logger.error(f"Error detallado: {fv_error}")
        import traceback
        traceback.print_exc()

except Exception as e:
    print(f"❌ Error al recuperar el feature group: {e}")
    logger.error(f"Error detallado: {e}")
    import traceback
    traceback.print_exc()

✅ Feature group encontrado: bolleria_predicciones_feature_group (versión 1)

Datos actuales en el feature group:
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.04s) 
                 week_start  predicted_base_imponible
0 2025-08-11 00:00:00+00:00               1310.636108

Creando feature view...
Creando nuevo feature view...
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1242272/fs/1224799/fv/bolleria_predicciones_feature_view/version/1
✅ Nuevo feature view creado: bolleria_predicciones_feature_view (versión 1)

Verificando datos en el feature view:
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
                 week_start  predicted_base_imponible
0 2025-08-11 00:00:00+00:00               1310.636108


# Visualización de las predicciones

Una vez tenemos las predicciones almacenadas en el feature store, podemos visualizarlas para comprender mejor las tendencias.

In [55]:
# Visualización de datos históricos y predicciones
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# 1. Obtener datos históricos del feature view original
try:
    # Obtenemos los datos históricos
    fv_historico = feature_store.get_feature_view(name=FV_NAME, version=FV_VERSION)
    df_historico = fv_historico.get_batch_data()
    df_historico = df_historico.sort_values('week_start')
    
    # 2. Obtenemos las predicciones del feature view de predicciones
    fv_pred = feature_store.get_feature_view(name=PRED_FV_NAME, version=PRED_FV_VERSION)
    df_pred = fv_pred.get_batch_data()
    df_pred = df_pred.sort_values('week_start')
    
    # 3. Preparar datos para visualización
    # Convertir a formato datetime si es necesario
    if not pd.api.types.is_datetime64_any_dtype(df_historico['week_start']):
        df_historico['week_start'] = pd.to_datetime(df_historico['week_start'])
    
    if not pd.api.types.is_datetime64_any_dtype(df_pred['week_start']):
        df_pred['week_start'] = pd.to_datetime(df_pred['week_start'])
    
    # 4. Visualización con Plotly
    fig = go.Figure()
    
    # Añadir datos históricos
    fig.add_trace(go.Scatter(
        x=df_historico['week_start'],
        y=df_historico['base_imponible'],
        mode='lines+markers',
        name='Histórico',
        line=dict(color='blue', width=2)
    ))
    
    # Añadir predicciones
    fig.add_trace(go.Scatter(
        x=df_pred['week_start'],
        y=df_pred['predicted_base_imponible'],
        mode='lines+markers',
        name='Predicciones',
        line=dict(color='red', width=2, dash='dash')
    ))
    
    # Configurar diseño
    fig.update_layout(
        title='Ventas Históricas y Predicciones para Bollería',
        xaxis_title='Fecha',
        yaxis_title='Base Imponible (€)',
        legend_title='Tipo de Dato',
        height=600,
        template='plotly_white'
    )
    
    # Mostrar gráfico
    fig.show()
    
except Exception as e:
    print(f"❌ Error al visualizar los datos: {e}")
    import traceback
    traceback.print_exc()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.69s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.74s) 


## Inferencia regresiva (opcional)

In [150]:
import numpy as np
from datetime import timedelta

print("=== PIPELINE DE INFERENCIA AUTOREGRESIVA ===")

# 1. Preparar datos base
ultimo_lunes = ts_df['week_start'].max()
last_row = ts_df.iloc[-1].copy()
feature_names = model.feature_names_in_

print(f"Última fecha histórica: {ultimo_lunes}")
print("Valores iniciales de lags:")
for lag in [1, 2, 3, 52]:
    col_name = f'base_imponible_lag{lag}'
    print(f"  {col_name}: {last_row[col_name]:.2f}")

# 2. Pipeline de predicción autoregresiva
predicciones_anteriores = []
features_pred = []

for semana in range(1, 5):  # 4 semanas futuras
    fecha = ultimo_lunes + timedelta(days=7 * semana)
    nueva_fila = last_row.copy()
    nueva_fila['week_start'] = fecha
    
    print(f"\n--- Semana {semana} ({fecha.date()}) ---")
    
    # Actualizar lags con predicciones anteriores
    for lag in [1, 2, 3, 52]:
        col_name = f'base_imponible_lag{lag}'
        valor_anterior = nueva_fila[col_name]
        
        if lag <= len(predicciones_anteriores):
            # Usar predicción anterior como lag
            nuevo_valor = predicciones_anteriores[-lag]
            nueva_fila[col_name] = nuevo_valor
            print(f"  {col_name}: {valor_anterior:.2f} -> {nuevo_valor:.2f}")
        else:
            # Mantener valor histórico
            print(f"  {col_name}: {valor_anterior:.2f} (histórico)")
    
    # Hacer predicción
    X = nueva_fila[feature_names].values.reshape(1, -1)
    prediccion = model.predict(X)[0]
    print(f"  Predicción: {prediccion:.2f}")
    
    # Guardar para siguiente iteración
    predicciones_anteriores.append(prediccion)
    nueva_fila['predicted_base_imponible'] = prediccion
    features_pred.append(nueva_fila.copy())

# 3. Crear DataFrame de resultados
results = pd.DataFrame(features_pred)[['week_start', 'predicted_base_imponible']]
fecha_limite = pd.Timestamp('2025-08-11 00:00:00+00:00')
results = results[results['week_start'] > fecha_limite].reset_index(drop=True)

print("\n=== RESULTADOS FINALES ===")
print(results)

=== PIPELINE DE INFERENCIA AUTOREGRESIVA ===
Última fecha histórica: 2025-07-28 00:00:00+00:00
Valores iniciales de lags:
  base_imponible_lag1: 1408.04
  base_imponible_lag2: 1318.98
  base_imponible_lag3: 1299.57
  base_imponible_lag52: 1714.05

--- Semana 1 (2025-08-04) ---
  base_imponible_lag1: 1408.04 (histórico)
  base_imponible_lag2: 1318.98 (histórico)
  base_imponible_lag3: 1299.57 (histórico)
  base_imponible_lag52: 1714.05 (histórico)
  Predicción: 1310.64

--- Semana 2 (2025-08-11) ---
  base_imponible_lag1: 1408.04 -> 1310.64
  base_imponible_lag2: 1318.98 (histórico)
  base_imponible_lag3: 1299.57 (histórico)
  base_imponible_lag52: 1714.05 (histórico)
  Predicción: 1329.65

--- Semana 3 (2025-08-18) ---
  base_imponible_lag1: 1408.04 -> 1329.65
  base_imponible_lag2: 1318.98 -> 1310.64
  base_imponible_lag3: 1299.57 (histórico)
  base_imponible_lag52: 1714.05 (histórico)
  Predicción: 1329.65

--- Semana 4 (2025-08-25) ---
  base_imponible_lag1: 1408.04 -> 1329.65
  bas

# Automatización de predicciones semanales

Definimos una función para automatizar el proceso completo de predicción semanal y su almacenamiento en Hopsworks.

In [ ]:
# Función para automatizar la predicción semanal y su almacenamiento
def prediccion_semanal_automatizada(
    proyecto_hopsworks,
    nombre_fg="bolleria_predicciones_feature_group",
    version_fg=1,
    nombre_fv="bolleria_predicciones_feature_view",
    version_fv=1,
    nombre_modelo="fleca_bolleria_predictor_next_week",
    version_modelo=1
):
    """
    Automatiza el proceso completo de:
    1. Obtener datos históricos
    2. Preparar features
    3. Predecir siguiente semana
    4. Almacenar predicción en feature store
    
    Args:
        proyecto_hopsworks: Proyecto de Hopsworks ya conectado
        nombre_fg: Nombre del feature group para almacenar predicciones
        version_fg: Versión del feature group
        nombre_fv: Nombre del feature view
        version_fv: Versión del feature view
        nombre_modelo: Nombre del modelo en el registry
        version_modelo: Versión del modelo
        
    Returns:
        DataFrame con la predicción realizada
    """
    import logging
    from datetime import datetime, timedelta
    import pandas as pd
    import numpy as np
    from src.model import transformar_features_target
    
    # Configuración de logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger('prediccion_semanal')
    
    try:
        # 1. Conexiones a Hopsworks
        feature_store = proyecto_hopsworks.get_feature_store()
        model_registry = proyecto_hopsworks.get_model_registry()
        
        logger.info("Conexión a Hopsworks establecida")
        
        # 2. Obtener datos históricos
        feature_view = feature_store.get_feature_view(name="times_series_bolleria_feature_view", version=1)
        ts_df = feature_view.get_batch_data()
        ts_df = ts_df.sort_values('week_start').reset_index(drop=True)
        
        logger.info(f"Datos históricos obtenidos: {len(ts_df)} registros")
        
        # 3. Preparar features
        ts_df = transformar_features_target(
            ts_df,
            lags_list=[1, 2, 3, 52], 
            columna_target='base_imponible',
            cols_exogenas=['is_easter', 'is_summer_peak'],
            periodos_adelante=1,
            eliminar_nulos=True,
            return_format='dataframe'
        )
        
        logger.info("Features preparados correctamente")
        
        # 4. Cargar modelo
        model = model_registry.get_model(name=nombre_modelo, version=version_modelo)
        from pathlib import Path
        import joblib
        model_dir = model.download()
        model_obj = joblib.load(Path(model_dir) / 'xgboost_hopsworks.pkl')
        
        logger.info(f"Modelo {nombre_modelo} v{version_modelo} cargado correctamente")
        
        # 5. Obtener la última fecha y predecir siguiente semana
        ultimo_lunes = ts_df['week_start'].max()
        fecha_siguiente = ultimo_lunes + timedelta(days=7)
        
        last_row = ts_df.iloc[-1].copy()
        last_row['week_start'] = fecha_siguiente
        
        feature_names = model_obj.feature_names_in_
        X = last_row[feature_names].values.reshape(1, -1)
        prediccion = model_obj.predict(X)[0]
        
        # 6. Crear DataFrame de resultados
        resultado = pd.DataFrame({
            'week_start': [fecha_siguiente],
            'predicted_base_imponible': [prediccion]
        })
        
        logger.info(f"Predicción para {fecha_siguiente.date()}: {prediccion:.2f}")
        
        # 7. Almacenar en feature store
        fg_pred = None
        
        # Intentar obtener feature group existente
        try:
            fg_pred = feature_store.get_feature_group(name=nombre_fg, version=version_fg)
            logger.info(f"Feature group '{nombre_fg}' existente encontrado")
        except:
            # Si no existe, crearlo
            fg_pred = feature_store.create_feature_group(
                name=nombre_fg,
                version=version_fg,
                description="Predicciones semanales de base_imponible para bolleria",
                primary_key=["week_start"],
                online_enabled=False,
                statistics_config={"enabled": True},
                event_time='week_start'
            )
            logger.info(f"Feature group '{nombre_fg}' creado correctamente")
        
        # Insertar predicción
        if fg_pred is not None:
            fg_pred.insert(resultado, write_options={'wait_for_job': True})
            logger.info("Predicción insertada en feature group")
            
            # 8. Crear feature view si no existe
            try:
                fv_pred = feature_store.get_feature_view(name=nombre_fv, version=version_fv)
                logger.info(f"Feature view '{nombre_fv}' existente encontrado")
            except:
                query = fg_pred.select_all()
                fv_pred = feature_store.create_feature_view(
                    name=nombre_fv,
                    version=version_fv,
                    query=query,
                    description="Vista de predicciones semanales de bolleria"
                )
                logger.info(f"Feature view '{nombre_fv}' creado correctamente")
        
        return resultado
        
    except Exception as e:
        logger.error(f"Error en predicción automatizada: {e}")
        import traceback
        traceback.print_exc()
        return None

# Ejemplo de uso (comentado para no ejecutarlo accidentalmente)
"""
# Para ejecutar una predicción automatizada, descomenta y ejecuta:
nueva_prediccion = prediccion_semanal_automatizada(
    proyecto_hopsworks=project
)
print("Nueva predicción realizada:")
print(nueva_prediccion)
"""

# Conclusiones

En este notebook hemos implementado un pipeline completo de inferencia para predecir ventas semanales de bollería:

1. **Conexión a Hopsworks**: Nos conectamos al feature store y al model registry.
2. **Carga de datos históricos**: Obtenemos datos del feature view existente.
3. **Preparación de features**: Transformamos los datos añadiendo lags y variables exógenas.
4. **Predicción**: Utilizamos el modelo XGBoost entrenado para predecir la próxima semana.
5. **Almacenamiento en Feature Store**: Guardamos las predicciones en un feature group específico.
6. **Feature View**: Creamos un feature view para acceder fácilmente a las predicciones.
7. **Visualización**: Representamos gráficamente los datos históricos y las predicciones.
8. **Automatización**: Implementamos una función para automatizar todo el proceso.

Este pipeline se puede programar para ejecutarse semanalmente, generando predicciones automáticas que se almacenan en Hopsworks para su posterior visualización y análisis.